## Работа с API

In [1]:
import requests
import pandas as pd


Получить информацию обо всех книгах и сохранить в pd.DataFrame

In [2]:
BASE_URL = "https://www.anapioficeandfire.com/api"

resp_books = requests.get(f"{BASE_URL}/books")
resp_books.raise_for_status()

books_data = resp_books.json()
books_df = pd.DataFrame(books_data)

books_df.head()


,url,name,isbn,authors,numberOfPages,publisher,country,mediaType,released,characters,povCharacters
0,https://www.anapioficeandfire.com/api/books/1,A Game of Thrones,978-0553103540,[George R. R. Martin],694,Bantam Books,United States,Hardcover,1996-08-01T00:00:00,[https://www.anapioficeandfire.com/api/charact...,[https://www.anapioficeandfire.com/api/charact...
1,https://www.anapioficeandfire.com/api/books/2,A Clash of Kings,978-0553108033,[George R. R. Martin],768,Bantam Books,United States,Hardback,1999-02-02T00:00:00,[https://www.anapioficeandfire.com/api/charact...,[https://www.anapioficeandfire.com/api/charact...
2,https://www.anapioficeandfire.com/api/books/3,A Storm of Swords,978-0553106633,[George R. R. Martin],992,Bantam Books,United States,Hardcover,2000-10-31T00:00:00,[https://www.anapioficeandfire.com/api/charact...,[https://www.anapioficeandfire.com/api/charact...
3,https://www.anapioficeandfire.com/api/books/4,The Hedge Knight,978-0976401100,[George R. R. Martin],164,Dabel Brothers Publishing,United States,GraphicNovel,2005-03-09T00:00:00,[https://www.anapioficeandfire.com/api/charact...,[https://www.anapioficeandfire.com/api/charact...
4,https://www.anapioficeandfire.com/api/books/5,A Feast for Crows,978-0553801507,[George R. R. Martin],784,Bantam Books,United Status,Hardcover,2005-11-08T00:00:00,[https://www.anapioficeandfire.com/api/charact...,[https://www.anapioficeandfire.com/api/charact...


Получить информацию обо всех домах Вестероса (Houses) и сохранить в pd.DataFrame

In [3]:
def load_all_houses(page_size: int = 50) -> pd.DataFrame:
    houses = []
    page = 1

    while True:
        resp = requests.get(
            f"{BASE_URL}/houses",
            params={"page": page, "pageSize": page_size}
        )
        resp.raise_for_status()
        data = resp.json()

        if not data:
            break

        houses.extend(data)
        page += 1

    return pd.DataFrame(houses)

houses_df = load_all_houses()
houses_df.head(), len(houses_df)


(                                              url  \
 0  https://www.anapioficeandfire.com/api/houses/1   
 1  https://www.anapioficeandfire.com/api/houses/2   
 2  https://www.anapioficeandfire.com/api/houses/3   
 3  https://www.anapioficeandfire.com/api/houses/4   
 4  https://www.anapioficeandfire.com/api/houses/5   
 
                           name           region  \
 0                 House Algood  The Westerlands   
 1  House Allyrion of Godsgrace            Dorne   
 2                  House Amber        The North   
 3                House Ambrose        The Reach   
 4   House Appleton of Appleton        The Reach   
 
                                           coatOfArms            words titles  \
 0  A golden wreath, on a blue field with a gold b...                      []   
 1          Gyronny Gules and Sable, a hand couped Or  No Foe May Pass     []   
 2                                                                         []   
 3                             Or, s

Получить информацию обо всех домах Вестероса, у которых есть девиз, и сохранить в pd.DataFrame

In [4]:
def load_houses_with_words(page_size: int = 50) -> pd.DataFrame:
    houses = []
    page = 1

    while True:
        resp = requests.get(
            f"{BASE_URL}/houses",
            params={
                "page": page,
                "pageSize": page_size,
                "hasWords": "true",
            },
        )
        resp.raise_for_status()
        data = resp.json()

        if not data:
            break

        houses.extend(data)
        page += 1

    return pd.DataFrame(houses)

houses_with_words_df = load_houses_with_words()
houses_with_words_df[["name", "region", "words"]].head(), len(houses_with_words_df)


(                             name          region                  words
 0     House Allyrion of Godsgrace           Dorne        No Foe May Pass
 1                   House Ambrose       The Reach          Never Resting
 2        House Arryn of the Eyrie        The Vale       As High as Honor
 3        House Ashford of Ashford       The Reach  Our Sun Shines Bright
 4  House Baratheon of Storm's End  The Stormlands       Ours is the Fury,
 68)

## Работа с БД

Установить библиотеку psycopg2

In [6]:
pip install psycopg2-binary pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 44.7 MB/s eta 0:00:00


In [7]:
import psycopg2
import pandas as pd


Подключиться к БД

In [8]:
conn = psycopg2.connect(
    host="hh-pgsql-public.ebi.ac.uk",
    port=5432,
    database="pfmegrnargs",
    user="reader",
    password="NWDMCE5xdipIjRrp",
)

cursor = conn.cursor()

Получите 10 строк из таблицы rnc_database
Сохраните полученные данные в pd.DataFrame

In [9]:
cursor.execute("SELECT * FROM rnc_database LIMIT 10;")
rows = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]

rnc_all_df = pd.DataFrame(rows, columns=columns)
rnc_all_df


,id,timestamp,userstamp,descr,current_release,full_descr,alive,for_release,display_name,project_id,avg_length,min_length,max_length,num_sequences,num_organisms,description,url,example,reference
0,21,2017-05-02,RNACEN,NONCODE,146,NONCODE,Y,,NONCODE,,1130.0,201.0,244296.0,234669,7,is an integrated knowledge database dedicated ...,http://www.noncode.org/,"[{'upi': 'URS000019B796', 'taxid': 9606}, {'up...",[{'title': 'NONCODE 2016: an informative and v...
1,5,2017-05-17,RNACEN,VEGA,98,VEGA,N,,VEGA,PRJEB4568,NaN,NaN,NaN,0,0,is a repository for high-quality gene models p...,http://vega.sanger.ac.uk/,"[{'upi': 'URS00000B15DA', 'taxid': 9606}, {'up...",[{'title': 'The GENCODE v7 catalog of human lo...
2,26,2017-05-01,RNACEN,GENCODE,450,GENCODE,N,,GENCODE,,889.0,32.0,205012.0,47677,2,produces high quality reference gene annotatio...,http://gencodegenes.org/,"[{'upi': 'URS00000B15DA', 'taxid': 9606}, {'up...",[{'title': 'GENCODE: the reference human genom...
3,1,2017-05-01,RNACEN,ENA,968,ENA,Y,,ENA,,412.0,10.0,900074.0,12086180,814855,provides a comprehensive record of the world's...,https://www.ebi.ac.uk/ena/browser/,"[{'upi': 'URS00002D0E0C', 'taxid': 10090}, {'u...",[{'title': 'The European Nucleotide Archive in...
4,14,2017-05-01,RNACEN,TAIR,982,TAIR,Y,,TAIR,PRJ_TAIR,384.0,19.0,6227.0,4406,1,is a database of genetic and molecular biology...,http://www.arabidopsis.org/,"[{'upi': 'URS0000591E4F', 'taxid': 3702}, {'up...",[{'title': 'The Arabidopsis Information Resour...
5,9,2017-05-01,RNACEN,REFSEQ,969,RefSeq,Y,,RefSeq,,703.0,15.0,91667.0,120355,22524,"is a comprehensive, integrated, non-redundant,...",http://www.ncbi.nlm.nih.gov/refseq/,"[{'upi': 'URS000075A3E5', 'taxid': 10090}, {'u...",[{'title': 'RefSeq: an update on mammalian ref...
6,41,2017-05-01,RNACEN,GENECARDS,978,MalaCards,Y,,GeneCards,,1292.0,16.0,347561.0,425357,1,"is a searchable, integrative database that pro...",https://www.genecards.org/,"[{'upi': 'URS0000EBFCE3', 'taxid': 9606}, {'up...",[{'title': 'The GeneCards Suite: From Gene Dat...
7,10,2017-05-01,RNACEN,RDP,85,RDP,Y,,RDP,,1536.0,1337.0,1600.0,4779,2487,"provides quality-controlled, aligned and annot...",http://rdp.cme.msu.edu/,"[{'upi': 'URS0000434740', 'taxid': 338963}, {'...",[{'title': 'Ribosomal Database Project: data a...
8,20,2017-05-01,RNACEN,LNCIPEDIA,935,LNCipedia,Y,,LNCipedia,,1534.0,200.0,152544.0,126876,1,is a comprehensive compendium of human long no...,http://www.lncipedia.org/,"[{'upi': 'URS000081175C', 'taxid': 9606}, {'up...",[{'title': 'An update on LNCipedia: a database...
9,15,2017-05-02,RNACEN,WORMBASE,970,WormBase,Y,,WormBase,PRJNA13758,171.0,17.0,84141.0,25550,1,"curates, stores and displays genomic and genet...",http://www.wormbase.org/,"[{'upi': 'URS000022A09E', 'taxid': 6239}, {'up...","[{'title': 'WormBase 2012: more genomes, more ..."


Получите значения столбцов display_name, num_sequences, num_organisms, , url для 10 строк из таблицы rnc_database

In [10]:
query = """
    SELECT display_name, num_sequences, num_organisms, url
    FROM rnc_database
    LIMIT 10;
"""
cursor.execute(query)
rows = cursor.fetchall()

rnc_subset_df = pd.DataFrame(
    rows,
    columns=["display_name", "num_sequences", "num_organisms", "url"]
)

rnc_subset_df


,display_name,num_sequences,num_organisms,url
0,NONCODE,234669,7,http://www.noncode.org/
1,VEGA,0,0,http://vega.sanger.ac.uk/
2,GENCODE,47677,2,http://gencodegenes.org/
3,ENA,12086180,814855,https://www.ebi.ac.uk/ena/browser/
4,TAIR,4406,1,http://www.arabidopsis.org/
5,RefSeq,120355,22524,http://www.ncbi.nlm.nih.gov/refseq/
6,GeneCards,425357,1,https://www.genecards.org/
7,RDP,4779,2487,http://rdp.cme.msu.edu/
8,LNCipedia,126876,1,http://www.lncipedia.org/
9,WormBase,25550,1,http://www.wormbase.org/


In [11]:
cursor.close()
conn.close()
